<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/Beredsen_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
import RFE_Glioblastomav2 as rfev2

In [2]:
dataset = pd.read_excel('/content/Beredsen.xlsx')

In [3]:
dataset.columns

Index(['age', 'biopsy_debulking', 'KPS', 'adjuvant_treatment', 'SVZ status',
       'survival ', 'censoring', 'Unnamed: 7', 'Variables'],
      dtype='object')

In [4]:
dataset.drop(columns = ['Unnamed: 7', 'Variables'], inplace = True)

In [5]:
dataset.head()

,age,biopsy_debulking,KPS,adjuvant_treatment,SVZ status,survival,censoring
0,82,1.0,1.0,1.0,0.0,176.0,1.0
1,67,1.0,1.0,2.0,0.0,362.0,0.0
2,70,1.0,1.0,2.0,0.0,433.0,1.0
3,49,2.0,1.0,1.0,0.0,117.0,1.0
4,78,2.0,1.0,1.0,0.0,326.0,1.0


In [6]:
dataset = dataset.rename(columns={'censoring': 'death', 'survival ' : 'OS_time'})

In [29]:
dataset.death.describe()

count    647.000000
mean       0.768161
std        0.422333
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: death, dtype: float64

In [7]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [8]:
imp_mean = IterativeImputer(random_state=0)

In [9]:
data = imp_mean.fit_transform(dataset)

In [10]:
data = pd.DataFrame(data=data, columns= dataset.columns,)

In [11]:
data.corr(method = 'pearson')

,age,biopsy_debulking,KPS,adjuvant_treatment,SVZ status,OS_time,death
age,1.000000,-0.082257,-0.161910,-0.029705,0.061247,-0.362111,0.173820
biopsy_debulking,-0.082257,1.000000,0.148640,-0.013053,-0.192748,0.363858,-0.144109
KPS,-0.161910,0.148640,1.000000,0.041573,-0.105957,0.297927,-0.127053
adjuvant_treatment,-0.029705,-0.013053,0.041573,1.000000,-0.015329,0.054873,-0.038990
SVZ status,0.061247,-0.192748,-0.105957,-0.015329,1.000000,-0.260603,0.110869
OS_time,-0.362111,0.363858,0.297927,0.054873,-0.260603,1.000000,-0.476101
death,0.173820,-0.144109,-0.127053,-0.038990,0.110869,-0.476101,1.000000


## Classification scores

In [20]:
X = data.drop(columns=['death', 'OS_time'], axis = 1)
y = data['death']

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [21]:
clf = RandomForestClassifier(random_state=0)


In [24]:
mttavg= []
favg= []
accuracyavg = []
tpavg = []
tnavg = []
rocavg = []
praucavg = []




for x in range(100): 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,)
  clf.fit(X_train, y_train)
  #That's for the ranking, now we gotta do for the validation set and also for all the metrics
  mttaverage = matthews_corrcoef(y_test, clf.predict(X_test))
  mttavg.append(mttaverage)
   #f1 score
  faverage = f1_score(y_test, clf.predict(X_test))
  favg.append(faverage)
  #accuracy score
  accuracyaverage = accuracy_score(y_test, clf.predict(X_test))
  accuracyavg.append(accuracyaverage)
  #TP and TN rate 
  tn, fp, fn, tp = confusion_matrix(y_test, clf.predict(X_test), labels =[0,1]).ravel()
  tprate =  tp / (tp + fn)
  tnrate = tn / (tn + fp)
  tpavg.append(tprate)
  tnavg.append(tnrate)
  # ROC AUC 
  roc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
  rocavg.append(roc)
  #Precision recall area under de curve
  precision, recall, _thresholds = metrics.precision_recall_curve(y_test, clf.predict_proba(X_test)[:, 1])
  prauc = metrics.auc(recall, precision)     
  praucavg.append(prauc)


mttavgmean = np.mean(np.array(mttavg))
favgmean = np.mean(np.array(favg))
accuracyavgmean = np.mean(np.array(accuracyavg))
tpavgmean = np.mean(np.array(tpavg))
tnavgmean = np.mean(np.array(tnavg))
rocavgmean = np.mean(np.array(rocavg))
praucavgmean = np.mean(np.array(praucavg))
metricss = np.array([mttavgmean, favgmean, accuracyavgmean, tpavgmean, tnavgmean, rocavgmean, praucavgmean])
metricss = metricss.reshape(1,7)
metric_final = pd.DataFrame(data = metricss, columns = ['matthewscore', 'f1-score', 'Accuracy', 'True Positives', 'True Negatives', 'ROC AUC', 'PR AUC'])

In [25]:
metric_final

,matthewscore,f1-score,Accuracy,True Positives,True Negatives,ROC AUC,PR AUC
0,0.056471,0.809418,0.695641,0.841867,0.208222,0.598024,0.833387


### XgBoost 

In [30]:
xgb = xgb.XGBClassifier(random_state=42)

In [31]:
mttavg= []
favg= []
accuracyavg = []
tpavg = []
tnavg = []
rocavg = []
praucavg = []




for x in range(100): 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
  xgb.fit(X_train, y_train)
  #That's for the ranking, now we gotta do for the validation set and also for all the metrics
  mttaverage = matthews_corrcoef(y_test,xgb.predict(X_test))
  mttavg.append(mttaverage)
   #f1 score
  faverage = f1_score(y_test,xgb.predict(X_test))
  favg.append(faverage)
  #accuracy score
  accuracyaverage = accuracy_score(y_test,xgb.predict(X_test))
  accuracyavg.append(accuracyaverage)
  #TP and TN rate 
  tn, fp, fn, tp = confusion_matrix(y_test,xgb.predict(X_test), labels =[0,1]).ravel()
  tprate =  tp / (tp + fn)
  tnrate = tn / (tn + fp)
  tpavg.append(tprate)
  tnavg.append(tnrate)
  # ROC AUC 
  roc = roc_auc_score(y_test, xgb.predict_proba(X_test)[:, 1])
  rocavg.append(roc)
  #Precision recall area under de curve
  precision, recall, _thresholds = metrics.precision_recall_curve(y_test, xgb.predict_proba(X_test)[:, 1])
  prauc = metrics.auc(recall, precision)     
  praucavg.append(prauc)

  
mttavgmean = np.mean(np.array(mttavg))
favgmean = np.mean(np.array(favg))
accuracyavgmean = np.mean(np.array(accuracyavg))
tpavgmean = np.mean(np.array(tpavg))
tnavgmean = np.mean(np.array(tnavg))
rocavgmean = np.mean(np.array(rocavg))
praucavgmean = np.mean(np.array(praucavg))
metricss = np.array([mttavgmean, favgmean, accuracyavgmean, tpavgmean, tnavgmean, rocavgmean, praucavgmean])
metricss = metricss.reshape(1,7)
metric_final = pd.DataFrame(data = metricss, columns = ['matthewscore', 'f1-score', 'Accuracy', 'True Positive rate', 'True Negative rate', 'ROC AUC', 'PR AUC'])
  

In [32]:
metric_final

,matthewscore,f1-score,Accuracy,True Positive rate,True Negative rate,ROC AUC,PR AUC
0,0.071818,0.84845,0.743846,0.93497,0.110531,0.634671,0.851623


### KNN

In [33]:
knn = KNeighborsClassifier()

In [34]:
mttavg= []
favg= []
accuracyavg = []
tpavg = []
tnavg = []
rocavg = []
praucavg = []




for x in range(100): 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
  knn.fit(X_train, y_train)
  #That's for the ranking, now we gotta do for the validation set and also for all the metrics
  mttaverage = matthews_corrcoef(y_test,knn.predict(X_test))
  mttavg.append(mttaverage)
   #f1 score
  faverage = f1_score(y_test,knn.predict(X_test))
  favg.append(faverage)
  #accuracy score
  accuracyaverage = accuracy_score(y_test,knn.predict(X_test))
  accuracyavg.append(accuracyaverage)
  #TP and TN rate 
  tn, fp, fn, tp = confusion_matrix(y_test,knn.predict(X_test), labels =[0,1]).ravel()
  tprate =  tp / (tp + fn)
  tnrate = tn / (tn + fp)
  tpavg.append(tprate)
  tnavg.append(tnrate)
  # ROC AUC 
  roc = roc_auc_score(y_test, knn.predict_proba(X_test)[:, 1])
  rocavg.append(roc)
  #Precision recall area under de curve
  precision, recall, _thresholds = metrics.precision_recall_curve(y_test, knn.predict_proba(X_test)[:, 1])
  prauc = metrics.auc(recall, precision)     
  praucavg.append(prauc)

mttavgmean = np.mean(np.array(mttavg))
favgmean = np.mean(np.array(favg))
accuracyavgmean = np.mean(np.array(accuracyavg))
tpavgmean = np.mean(np.array(tpavg))
tnavgmean = np.mean(np.array(tnavg))
rocavgmean = np.mean(np.array(rocavg))
praucavgmean = np.mean(np.array(praucavg))
metricss = np.array([mttavgmean, favgmean, accuracyavgmean, tpavgmean, tnavgmean, rocavgmean, praucavgmean])
metricss = metricss.reshape(1,7)
knn_metric_final = pd.DataFrame(data = metricss, columns = ['matthewscore', 'f1-score', 'Accuracy', 'True Positive rate', 'True Negative rate', 'ROC AUC', 'PR AUC'])

In [35]:
knn_metric_final

,matthewscore,f1-score,Accuracy,True Positive rate,True Negative rate,ROC AUC,PR AUC
0,0.056395,0.833661,0.72441,0.897975,0.145621,0.592646,0.851016


### Bagging Forest Balanced from Imbalanced learn

In [37]:
from imblearn.ensemble import BalancedBaggingClassifier
bbc = BalancedBaggingClassifier(n_estimators=100, random_state=0)


In [ ]:
mttavg= []
favg= []
accuracyavg = []
tpavg = []
tnavg = []
rocavg = []
praucavg = []




for x in range(100): 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,)
  bbc.fit(X_train, y_train)
  #That's for the ranking, now we gotta do for the validation set and also for all the metrics
  mttaverage = matthews_corrcoef(y_test, bbc.predict(X_test))
  mttavg.append(mttaverage)
   #f1 score
  faverage = f1_score(y_test, bbc.predict(X_test))
  favg.append(faverage)
  #accuracy score
  accuracyaverage = accuracy_score(y_test, bbc.predict(X_test))
  accuracyavg.append(accuracyaverage)
  #TP and TN rate 
  tn, fp, fn, tp = confusion_matrix(y_test, bbc.predict(X_test), labels =[0,1]).ravel()
  tprate =  tp / (tp + fn)
  tnrate = tn / (tn + fp)
  tpavg.append(tprate)
  tnavg.append(tnrate)
  # ROC AUC 
  roc = roc_auc_score(y_test, bbc.predict_proba(X_test)[:, 1])
  rocavg.append(roc)
  #Precision recall area under de curve
  precision, recall, _thresholds = metrics.precision_recall_curve(y_test, bbc.predict_proba(X_test)[:, 1])
  prauc = metrics.auc(recall, precision)     
  praucavg.append(prauc)


mttavgmean = np.mean(np.array(mttavg))
favgmean = np.mean(np.array(favg))
accuracyavgmean = np.mean(np.array(accuracyavg))
tpavgmean = np.mean(np.array(tpavg))
tnavgmean = np.mean(np.array(tnavg))
rocavgmean = np.mean(np.array(rocavg))
praucavgmean = np.mean(np.array(praucavg))
metricss = np.array([mttavgmean, favgmean, accuracyavgmean, tpavgmean, tnavgmean, rocavgmean, praucavgmean])
metricss = metricss.reshape(1,7)
metric_final = pd.DataFrame(data = metricss, columns = ['matthewscore', 'f1-score', 'Accuracy', 'True Positives', 'True Negatives', 'ROC AUC', 'PR AUC'])

In [39]:
metric_final

,matthewscore,f1-score,Accuracy,True Positives,True Negatives,ROC AUC,PR AUC
0,0.133561,0.726768,0.620615,0.658217,0.494993,0.616977,0.842339


## RFE with Matthew score as ranking system - Without Pogression 


In [40]:
import RFE_Glioblastomav2 as rfe
clf = RandomForestClassifier(random_state=0)

In [41]:
ranking = rfe.RankingRE(X, y, 100, clf)

In [42]:
rank, metrics_all_features, metrics_top2_features = ranking.ranking_by_matthew_punishment()

In [43]:
rank

,Categories,average-mtt-punishment,ranking,SD_of_matt_punishment
0,age,0.056889501554919404,1.0,0.0908661164323985
3,adjuvant_treatment,0.03260283718506046,2.0,0.05660381167501167
2,KPS,0.011776906784822056,3.0,0.040148147912240854
1,biopsy_debulking,0.010252407082233335,4.0,0.052332716360165586
4,SVZ status,-0.0109530061416281,5.0,0.06770415236698024


In [44]:
metrics_all_features

,matthewscore,f1-score,Accuracy,True Positives,True Negatives,ROC AUC,PR AUC
0,0.087761,0.816002,0.705981,0.849494,0.230008,0.586614,0.82248


### RFE with balanced bagging trees

In [45]:
bbc = BalancedBaggingClassifier(n_estimators=100, random_state=0)


In [46]:
ranking_balanced = rfe.RankingRE(X, y, 100, bbc)

In [ ]:
rank, metrics_all_features, metrics_top2_features = ranking_balanced.ranking_by_matthew_punishment()

In [48]:
rank

,Categories,average-mtt-punishment,ranking,SD_of_matt_punishment
3,adjuvant_treatment,0.038408325486875325,1.0,0.050013625403574326
1,biopsy_debulking,0.015502869568245783,2.0,0.04138271650795975
2,KPS,0.008825222898620444,3.0,0.03875298884654419
4,SVZ status,0.004053912340226398,4.0,0.05434686809419818
0,age,-0.07407705114369625,5.0,0.07626157739329288


In [49]:
metrics_all_features

,matthewscore,f1-score,Accuracy,True Positives,True Negatives,ROC AUC,PR AUC
0,0.132211,0.737195,0.630467,0.678506,0.471633,0.604075,0.826999


### Top 2 features classification balanced *bagged* Trees

In [50]:
bbc = BalancedBaggingClassifier(n_estimators=100, random_state=0)


In [52]:
X = data[['adjuvant_treatment', 'biopsy_debulking']]
y = data['death']

In [ ]:
mttavg= []
favg= []
accuracyavg = []
tpavg = []
tnavg = []
rocavg = []
praucavg = []




for x in range(100): 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,)
  bbc.fit(X_train, y_train)
  #That's for the ranking, now we gotta do for the validation set and also for all the metrics
  mttaverage = matthews_corrcoef(y_test, bbc.predict(X_test))
  mttavg.append(mttaverage)
   #f1 score
  faverage = f1_score(y_test, bbc.predict(X_test))
  favg.append(faverage)
  #accuracy score
  accuracyaverage = accuracy_score(y_test, bbc.predict(X_test))
  accuracyavg.append(accuracyaverage)
  #TP and TN rate 
  tn, fp, fn, tp = confusion_matrix(y_test, bbc.predict(X_test), labels =[0,1]).ravel()
  tprate =  tp / (tp + fn)
  tnrate = tn / (tn + fp)
  tpavg.append(tprate)
  tnavg.append(tnrate)
  # ROC AUC 
  roc = roc_auc_score(y_test, bbc.predict_proba(X_test)[:, 1])
  rocavg.append(roc)
  #Precision recall area under de curve
  precision, recall, _thresholds = metrics.precision_recall_curve(y_test, bbc.predict_proba(X_test)[:, 1])
  prauc = metrics.auc(recall, precision)     
  praucavg.append(prauc)


mttavgmean = np.mean(np.array(mttavg))
favgmean = np.mean(np.array(favg))
accuracyavgmean = np.mean(np.array(accuracyavg))
tpavgmean = np.mean(np.array(tpavg))
tnavgmean = np.mean(np.array(tnavg))
rocavgmean = np.mean(np.array(rocavg))
praucavgmean = np.mean(np.array(praucavg))
metricss = np.array([mttavgmean, favgmean, accuracyavgmean, tpavgmean, tnavgmean, rocavgmean, praucavgmean])
metricss = metricss.reshape(1,7)
metric_final = pd.DataFrame(data = metricss, columns = ['matthewscore', 'f1-score', 'Accuracy', 'True Positives', 'True Negatives', 'ROC AUC', 'PR AUC'])

In [55]:
metric_final

,matthewscore,f1-score,Accuracy,True Positives,True Negatives,ROC AUC,PR AUC
0,0.242892,0.720032,0.631333,0.620963,0.664539,0.666665,0.870869


### Top 2 features classification random forest


In [56]:
clf = RandomForestClassifier(random_state=0)

In [57]:
X = data[['adjuvant_treatment', 'age']]
y = data['death']

In [ ]:
mttavg= []
favg= []
accuracyavg = []
tpavg = []
tnavg = []
rocavg = []
praucavg = []




for x in range(100): 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,)
  bbc.fit(X_train, y_train)
  #That's for the ranking, now we gotta do for the validation set and also for all the metrics
  mttaverage = matthews_corrcoef(y_test, bbc.predict(X_test))
  mttavg.append(mttaverage)
   #f1 score
  faverage = f1_score(y_test, bbc.predict(X_test))
  favg.append(faverage)
  #accuracy score
  accuracyaverage = accuracy_score(y_test, bbc.predict(X_test))
  accuracyavg.append(accuracyaverage)
  #TP and TN rate 
  tn, fp, fn, tp = confusion_matrix(y_test, bbc.predict(X_test), labels =[0,1]).ravel()
  tprate =  tp / (tp + fn)
  tnrate = tn / (tn + fp)
  tpavg.append(tprate)
  tnavg.append(tnrate)
  # ROC AUC 
  roc = roc_auc_score(y_test, bbc.predict_proba(X_test)[:, 1])
  rocavg.append(roc)
  #Precision recall area under de curve
  precision, recall, _thresholds = metrics.precision_recall_curve(y_test, bbc.predict_proba(X_test)[:, 1])
  prauc = metrics.auc(recall, precision)     
  praucavg.append(prauc)


mttavgmean = np.mean(np.array(mttavg))
favgmean = np.mean(np.array(favg))
accuracyavgmean = np.mean(np.array(accuracyavg))
tpavgmean = np.mean(np.array(tpavg))
tnavgmean = np.mean(np.array(tnavg))
rocavgmean = np.mean(np.array(rocavg))
praucavgmean = np.mean(np.array(praucavg))
metricss = np.array([mttavgmean, favgmean, accuracyavgmean, tpavgmean, tnavgmean, rocavgmean, praucavgmean])
metricss = metricss.reshape(1,7)
metric_final = pd.DataFrame(data = metricss, columns = ['matthewscore', 'f1-score', 'Accuracy', 'True Positives', 'True Negatives', 'ROC AUC', 'PR AUC'])

In [59]:
metric_final

,matthewscore,f1-score,Accuracy,True Positives,True Negatives,ROC AUC,PR AUC
0,0.103571,0.678687,0.575385,0.588374,0.532185,0.598945,0.8297
